# Grab sequences from OTUs of interest

In [7]:
import sys, os, re, glob
from Bio import SeqIO, Entrez
from pandas import *
import numpy as np
#import entrez.direct
from cogent.app.fasttree import build_tree_from_alignment
from cogent import DNA, LoadSeqs

Entrez.email = "be68@cornell.edu"
nprocs = 3

In [13]:
FastaSource = '/home/be68/SorghumAMF/otu.fasta'
OTUlist = '/home/be68/SorghumAMF/Unknown_df.txt'

OTUlist is a dataframe exported from R phyloseq object.  Here is was exported as a character vector so the column heading is only "x"

In [74]:
df = pandas.read_csv(OTUlist, sep = "\t")
df.head()
df.OTU = df.OTU.astype(object)
df.dtypes

Kingdom    object
Phylum     object
Class      object
Order      object
Family     object
Genus      object
OTU        object
dtype: object

In [75]:
OTUcheck = df.OTU.tolist()
OTUcheck = [str(item) for item in OTUcheck]
OTUcheck[0:4]

['225', '284', '328', '335']

In [76]:
output = open("/home/be68/SorghumAMF/Unknown.fasta", "w")


for record in SeqIO.parse(open(FastaSource, "rU"), "fasta"):
    if record.id in OTUcheck:
        output.write(">"+record.id+"\n"+str(record.seq)+"\n")
    


In [77]:
%%bash 
head /home/be68/SorghumAMF/Unknown.fasta

>225
GACGGAGGGTGCAAACGTTGTTCGGAATCACTGGGCGTAAAGGGCGCGTAGGCGGACTTGTAAGTCGGGAGTGAAAACTCGGGGCTCAACCCCGAGCCTGCTTCCGAAACTGCGAGTCTTGAGGACCGGAGAGGAAAGTGGAATTCCTAGTGTAGCGGTGGAATGCGTAGATATTAGGAAGAACACCAGTGGCGAAGGCGACTTTCTGGACGGTTCCTGACGCTGAGGCGCGAAAGCTAGGGGAGCAAACAGG
>284
TACGGAGGGGGCGAGTGTTGTTCGGAATTATTGGGCGTAAAGGGCGCGCAGGCGGCTTGGCAAGTCGAGTGTGAAATCCCTTGGCTTAACTGAGGAATGGCGCTCGAAACTACCTCGCTAGAGGGCGGGAGAGGGAAGTGGAACTCTCGGTGTAGGGGTGAAATCTGTAGATATCGAGAGGAACACCGGTGGCGAAGGCGGCTTCCTGGCCCGCATCTGACGCTGAGGCGCGAAAGCGTGGGGAGCAAACAGG
>328
TACGAGGGGGGCAAATGTTGTTCGGAATCACTGGGCGTAAAGGGAGCGTAGGCGGACTTTTAAGTTAGGTGTTTAATCCCGGGGCCCAACCCCGGACAGGCACCTAAAACTGGAAGTCTTGAATAGTGGAGAGGACGGCGGAATTGGGGGTGTAGCGGTGGAATGCGTAGAGATCCCCAAGAACACCAGTTGCGAAGGCGGCCGTCTGGACACATATTGACGCTGAGGCTCGAAAGCATGGGGAGCAAACAGG
>335
AACGTAGGTGGCGAGCGTTATCCGGATTTACTGGGCGTAAAGGGTGTGCAGGCTGCCCCTCAAGTGGTGTATGAAAGGGCTCGGCTCAACCGGGCTAGGTTATGCCAGACTGAGGGGCTAGAGAGCGAGAGCGGGACGTGGAATTCCGGGTGTAGTGGTGAAATGCGTAGAGATCCGGAGGAACCCCAGAGGCGAAGGCGGCGTCCTGGCTCGCATCT

In [78]:
%%bash 
grep -c ">" /home/be68/SorghumAMF/Unknown.fasta

1541


# Trim seqs to those that blast to MRE reference strains

* re.fasta was prepared by downloading Moeniiplasma seqs from Desario paper
* See Hyphosphere/MRE_Tree.ipnyb

In [ ]:
## Make blast database

In [4]:
MREseqs = '/home/be68/Hyphosphere/data/MRE/ref.fasta'
Unknowns = '/home/be68/SorghumAMF/Unknown.fasta'
MREBlast = '/home/be68/SorghumAMF/Unknown-MRE_blast.txt'

In [5]:
!head /home/be68/Hyphosphere/data/MRE/ref.fasta

>MG052969.1 Uncultured Mollicutes bacterium clone Msp-GBAus27b 16S ribosomal RNA gene, partial sequence
AGAGTTTGATCCTGGCTCAGGATTAACGTTGGTAGTATGCCTAACACATGCAAGTCGAGCGGAAGAAAAT
AGTAATATTTTTTTTAGCGGCGAACGGGTGAGTAATGCTTATCTAACTTGCCGATTAGTGGGGAATAACT
ATGGGAAACCATTGCTAATACCGCATATTCGTTATTTAGTTAATGAAAGGAAATCGTAAGATTTTCGCTA
ATCGATAGGGATAAGTCGTATTAGCTAGTTGGTGAGGTAATGGCTCACCAAGGCAATGATGCGTAGCCAG
TCTGAGAGGATGAATGGTCACAAGGGAACTGAGACACGGTCCCTACTCCTACGGGAGGCAGCAGTGAGGA
ATTTTCCACAATGGGGGAAACCCTGATGGAGCAATACTACGTGAAGGATGAAGGTCTAAGGATTGTAAAC
TTCTGTTAGCGCGGAAGAAGGGTCGCAAGATCTTGACGGTACGCGCAGAGAAAGTCACGACTAACTACGT
GCCAGCAGTCGCGGTAAGACGTAGGTGGCGAACGTTATCCGGAATTATTGGGCGTAAAGGGTGCGTAGAT
GGCTTGGTAAGTTGGCGGTGGGAAATCGAGGCTCAACCTCGTGGAAGCCGGCAATACTACTAAGCTCGAG


In [12]:
%%bash -s "$MREseqs"
makeblastdb -in $1 -parse_seqids -dbtype nucl




Building a new DB, current time: 11/06/2018 16:00:01
New DB name:   /home/be68/Hyphosphere/data/MRE/ref.fasta
New DB title:  /home/be68/Hyphosphere/data/MRE/ref.fasta
Sequence type: Nucleotide
Deleted existing BLAST database with identical name.
Keep Linkouts: T
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 66 sequences in 0.0219531 seconds.


In [13]:
%%bash -s "$MREseqs" "$Unknowns" "$MREBlast"
blastn -db $1 -query $2 -out $3 -max_target_seqs 1 -outfmt 6

In [14]:
%%bash -s "$MREseqs" "$Unknowns" "$MREBlast"
head $3

225	KC441811.1	80.24	248	46	3	8	253	494	740	8e-49	  183
284	KC441811.1	78.12	256	50	5	1	253	488	740	5e-41	  158
363	KP763351.1	72.43	243	60	5	10	248	384	623	7e-15	71.3
403	KC441811.1	77.64	246	53	2	6	250	493	737	3e-38	  148
520	KC441811.1	78.78	245	52	0	6	250	493	737	3e-43	  165
683	KC441811.1	76.26	257	54	7	1	254	488	740	1e-32	  130
791	KC441811.1	79.52	249	47	4	8	254	494	740	5e-46	  174
887	KC441811.1	80.00	250	50	0	1	250	488	737	2e-49	  185
939	KC441811.1	77.73	256	51	5	1	253	488	740	2e-39	  152
991	KC441811.1	77.78	243	48	6	14	253	501	740	4e-37	  145


In [8]:
df = pandas.read_table(MREBlast, sep = "\t", header=None, 
                       names = ["OTU", "sseqid", "pident", "length", "mismatch", "gapopen", "qstart",
                                "qend", "sstart", "send", "evalue", "bitscore"])

df.head()

,OTU,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore
0,225,KC441811.1,80.24,248,46,3,8,253,494,740,8.000000e-49,183.0
1,284,KC441811.1,78.12,256,50,5,1,253,488,740,5.000000e-41,158.0
2,363,KP763351.1,72.43,243,60,5,10,248,384,623,7.000000e-15,71.3
3,403,KC441811.1,77.64,246,53,2,6,250,493,737,3.000000e-38,148.0
4,520,KC441811.1,78.78,245,52,0,6,250,493,737,3.000000e-43,165.0


### Filter to >85% match to MREs

In [9]:
df_mre = df.loc[(df['pident'] > 85) & (df['length'] > 240)]
df_mre

,OTU,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore
41,2872,FJ984707.1,99.21,254,2,0,1,254,383,636,1.000000e-131,459.0
44,3001,KP763351.1,94.09,254,15,0,1,254,375,628,6.000000e-110,387.0
63,4190,FJ984646.1,98.82,254,2,1,1,254,475,727,2.000000e-129,451.0
64,4322,KC441811.1,85.08,248,34,3,8,253,494,740,8.000000e-69,250.0
66,4442,FJ984657.1,95.31,256,12,0,1,256,498,753,5.000000e-116,407.0
71,4562,KP763336.1,93.28,253,17,0,1,253,409,661,5.000000e-106,374.0
90,5222,FJ984653.1,91.34,254,22,0,1,254,475,728,3.000000e-98,348.0
133,6562,KC441811.1,85.43,254,35,2,1,253,488,740,1.000000e-72,263.0
140,6712,FJ984635.1,89.58,259,21,5,1,256,421,676,5.000000e-91,324.0
147,6878,FJ984654.1,98.82,255,3,0,1,255,480,734,2.000000e-130,455.0


In [10]:
df_mre.shape

(41, 12)

### Grab seqs from fasta

In [11]:
OTUcheck = df_mre.OTU.tolist()
OTUcheck = [str(item) for item in OTUcheck]


In [17]:
output = open("/home/be68/SorghumAMF/NebraskaMRE_85percent.fasta", "w")

#FastaSource same as defined above
for record in SeqIO.parse(open(FastaSource, "rU"), "fasta"):
    if record.id in OTUcheck:
        output.write(">"+record.id+"\n"+str(record.seq)+"\n")

In [18]:
!head /home/be68/SorghumAMF/NebraskaMRE_85percent.fasta

>2872
GACATAGGTGGCGAACGTTATCCGGAATTACTGGGCGTAAAGAGTGCGTAGGTGGTTCAATAAGTTACTGGTGGGAAATCGGGGCTCAACCCCGTGGAAGTCAGTAATACTATTAAACTAGAGGGTCGGAGAGGTTAACGGAATTCTATGTGTAGCGGTAAAATGCTTAAATATATAGAGGAACACCAAACGGCGAAGGCAGTTAACTATTCGATTCCTGACGTTGAGGCACGAAAGCGTGGGGAGCAAGACGG
>3001
GACATAGGTGGCAAACGTTATCCGGAATTACTGGGCGTAAAGTGTGCGTAGGTGGTTGAATAAGTTACTGGTGGCAAATCGGGGCTCAACTTCGTGGAAGCCAGTAATACTATTCAACTCGAGGACCGGAGAGGTTAACGGAATTCTATGTGTAGCGGTAAAATGCTCAAATATATAGAGGAACATCAATCGGCGAAAGCAGTTAACTATTCGGTTCCTGACGTTGAGGCACGAAAGCGTGGGGAGCAAGACGG
>4190
GACATAGGTGGCGAACGTTATCCGGAATTACTGGGCGTAAAGAGTGCGTAGGTGGTTCAATAAGTTACTGGTGGCAAATCGGGGCCCAACCCCGTGGAAGTTGGTAATACTGTTGAACTAGAGGACCGGAGAGGTTAACGGAATTCTATGTGTAGCGGTAAAATGCTCAAATATATAGAGGAACACCAAACGGCGAAGGCAGTTAACTATTCGGTGCCTGACGTTGAGGCACGAAAGCGTGGGGAGCAAGACGG
>4322
TACAGGGGGTGCAAGCGTTATTCGGAATTATTGGGCGTAAAGAGCGCGTAGGCGGTCTATTAAGTCTGGTGTGAAAGCCCTGGGCTCAATCCAGGAAGTGCATTGGATACTGATAGACTAGAGTACAGGAGAGGTAGGCGGAATTCCAGGTGTAGAGGTGAAATTCGTAGATATCTGGAGGAACAACAGTGGCGAAGGCGGCCTACTGGAC

# Build tree with Nebraska and Desario sequences

## Use SSU align to align both datasets and mask to smaller region

In [19]:
NebraskaMREseqs = '/home/be68/SorghumAMF/NebraskaMRE_85percent.fasta'

In [23]:
workDir = '/home/be68/SorghumAMF/NebraskaMRE_85percent'
os.chdir(workDir)

In [24]:
%%bash -s "$NebraskaMREseqs"
ssu-prep -f -x -b 50 --rfonly --dna $1 ssu_aln 1

# _ssu-prep :: prepare SSU rRNA sequences for parallel ssu-align jobs
# SSU-ALIGN 0.1.1 (Feb 2016)
# Copyright (C) 2016 Howard Hughes Medical Institute
# Freely distributed under the BSD open source license.
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
# command: _ssu-prep -x -f -b 50 --dna --rfonly /home/be68/SorghumAMF/NebraskaMRE_85percent.fasta ssu_aln 1
# date:    Tue Nov 13 20:55:36 2018
#
# Validating input sequence file ... done.
#
# Preparing 1 ssu-align jobs ...
# Partitioning seqs with goal of equalizing total number of nucleotides per job ...
#
# output file name              description                                        
# ----------------------------  ---------------------------------------------------
  ssu_aln/NebraskaMRE_85percent.fasta.1  partition 1 FASTA sequence file (41 seqs; 10446 nt)
  ssu_aln.ssu-align.sh          shell script that will execute 1 ssu-align jobs
#
#################################################################

In [25]:
%%bash
./ssu_aln.ssu-align.sh

# Executing: ssu-align --merge 1 -b 50 --dna --rfonly ssu_aln/NebraskaMRE_85percent.fasta.1 ssu_aln/ssu_aln.1
# _ssu-align :: align SSU rRNA sequences
# SSU-ALIGN 0.1.1 (Feb 2016)
# Copyright (C) 2016 Howard Hughes Medical Institute
# Freely distributed under the BSD open source license.
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
# command: _ssu-align --merge 1 -b 50 --dna --rfonly ssu_aln/NebraskaMRE_85percent.fasta.1 ssu_aln/ssu_aln.1
# date:    Tue Nov 13 20:55:40 2018
#
# Validating input sequence file ... done.
#
# Stage 1: Determining SSU start/end positions and best-matching models...
#
# output file name            description                                
# --------------------------  -------------------------------------------
  ssu_aln.1.tab               locations/scores of hits defined by HMM(s)
  ssu_aln.1.bacteria.hitlist  list of sequences to align with bacteria CM
  ssu_aln.1.bacteria.fa            41 sequences to align with bacteria CM

In [26]:
!ssu-mask --dna --afa ssu_aln

# _ssu-mask :: mask SSU rRNA alignments
# SSU-ALIGN 0.1.1 (Feb 2016)
# Copyright (C) 2016 Howard Hughes Medical Institute
# Freely distributed under the BSD open source license.
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
# command: _ssu-mask --afa --dna ssu_aln
# date:    Tue Nov 13 20:55:52 2018
#
# Masking alignments based on posterior probabilities...
#
#                                                     mask    
#                                                 ------------
# file name                  in/out  type  #cols  incl.  excl.
# -------------------------  ------  ----  -----  -----  -----
  ssu_aln.bacteria.stk        input   aln   1582      -      -
  ssu_aln.bacteria.mask      output  mask   1582    229   1353
  ssu_aln.bacteria.mask.pdf  output   pdf   1582    229   1353
  ssu_aln.bacteria.mask.afa  output   aln    229      -      -
#
# All attempts to draw structure diagrams of masks were successful.
#
# List of executed commands saved 

## Align ref fasta

In [27]:
%%bash -s "$MREseqs"
ssu-prep -f -x -b 50 --rfonly --dna $1 ssr_aln 1

# _ssu-prep :: prepare SSU rRNA sequences for parallel ssu-align jobs
# SSU-ALIGN 0.1.1 (Feb 2016)
# Copyright (C) 2016 Howard Hughes Medical Institute
# Freely distributed under the BSD open source license.
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
# command: _ssu-prep -x -f -b 50 --dna --rfonly /home/be68/Hyphosphere/data/MRE/ref.fasta ssr_aln 1
# date:    Tue Nov 13 20:55:53 2018
#
# Validating input sequence file ... done.
#
# Preparing 1 ssu-align jobs ...
# Partitioning seqs with goal of equalizing total number of nucleotides per job ...
#
# output file name      description                                        
# --------------------  ---------------------------------------------------
  ssr_aln/ref.fasta.1   partition 1 FASTA sequence file (66 seqs; 82218 nt)
  ssr_aln.ssu-align.sh  shell script that will execute 1 ssu-align jobs
#
################################################################################
# To execute all 1 ssu-align jobs

In [28]:
%%bash
./ssr_aln.ssu-align.sh

# Executing: ssu-align --merge 1 -b 50 --dna --rfonly ssr_aln/ref.fasta.1 ssr_aln/ssr_aln.1
# _ssu-align :: align SSU rRNA sequences
# SSU-ALIGN 0.1.1 (Feb 2016)
# Copyright (C) 2016 Howard Hughes Medical Institute
# Freely distributed under the BSD open source license.
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
# command: _ssu-align --merge 1 -b 50 --dna --rfonly ssr_aln/ref.fasta.1 ssr_aln/ssr_aln.1
# date:    Tue Nov 13 20:55:53 2018
#
# Validating input sequence file ... done.
#
# Stage 1: Determining SSU start/end positions and best-matching models...
#
# output file name            description                                
# --------------------------  -------------------------------------------
  ssr_aln.1.tab               locations/scores of hits defined by HMM(s)
  ssr_aln.1.bacteria.hitlist  list of sequences to align with bacteria CM
  ssr_aln.1.bacteria.fa            66 sequences to align with bacteria CM
#
# Stage 2: Aligning each sequence

In [29]:
!ssu-mask -s ssu_aln/ssu_aln.bacteria.mask --dna --afa ssr_aln/

# _ssu-mask :: mask SSU rRNA alignments
# SSU-ALIGN 0.1.1 (Feb 2016)
# Copyright (C) 2016 Howard Hughes Medical Institute
# Freely distributed under the BSD open source license.
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
# command: _ssu-mask -s ssu_aln/ssu_aln.bacteria.mask --afa --dna ssr_aln/
# date:    Tue Nov 13 20:57:40 2018
#
# Masking alignments using pre-existing masks...
#
#                                                     mask    
#                                                 ------------
# file name                  in/out  type  #cols  incl.  excl.
# -------------------------  ------  ----  -----  -----  -----
  ssr_aln.bacteria.stk        input   aln   1582      -      -
  ssu_aln.bacteria.mask       input  mask   1582    229   1353
  ssr_aln.bacteria.mask.pdf  output   pdf   1582    229   1353
  ssr_aln.bacteria.mask.afa  output   aln    229      -      -
#
# All attempts to draw structure diagrams of masks were successful.
#
# List o

In [30]:
!cat ssr_aln/ssr_aln.bacteria.mask.afa ssu_aln/ssu_aln.bacteria.mask.afa > NebraskMREaln_for_tree.fasta

In [31]:
!tail NebraskMREaln_for_tree.fasta

>21321
-ACATAGGTGGCAAACGTTATCCGGAATTATTGGGCGTAAAGAGTGCGTAGGTGGTTTAA
TAAGTTACTGGTGGCTCAACCTCAGTAATACTGTTAAACTAGAGGGCCGGAGAGGTTAAC
GGAATGGTTCATGTAGGGGTAAAATCCATAAATATGAATTAGAACATCGGCGAAGGCAGT
TAACTATTCGACCTGACGTTGAGGCACGAAAGCGTGGGTAGCAA-----
>21850
-ACGTAGGTGGCGAACGTTATCCGGAATTATTGGGCGTAAAGGGTGCGTAGATGGTTTGA
TAAGTTAACGGTGGCCCAACCCCGGTAATACTGTCCGACTAGAGGGCAGGAGAGGTTAAC
GGAATGGTTCGTGGAGGGGTAAAATCCATAGATATGAACTAGAACACCGGCGAAGGCAGT
TAACTATTCTGCCTGACATTGAGGCACGAAAGCGTGGGGAGCAAA----


In [ ]:
#rename ref MREs

In [32]:
NewName = {"KP763319" : "MRE from Claroideoglomus etunicatum",
           "KP763328" : "MRE from Claroideoglomus etunicatum",
          "KP763360": "MRE from Claroideoglomus etunicatum",
           "KP763286" : "MRE from Claroideoglomus etunicatum",
           "FJ984689": "MRE from Claroideoglomus etunicatum",
           "KP763351" : "MRE from Claroideoglomus etunicatum",
           "FJ984694" : "MRE from Claroideoglomus etunicatum",
           "KP763291" : "MRE from Claroideoglomus etunicatum",
           "KP763316" : "MRE from Claroideoglomus etunicatum",
           "KP763336" : "MRE from Claroideoglomus etunicatum",
           "FJ984660" : "MRE from Claroideoglomus claroideum",
           "FJ984671" : "MRE from Claroideoglomus claroideum",
           "FJ984658" : "MRE from Claroideoglomus claroideum",
            "KP763388" : "MRE from Gigaspora albida", 
           "KP763383": "MRE from Gigaspora albida", 
             "KF378701":  "MRE from Gigaspora margarita",
           "KF378698":  "MRE from Gigaspora margarita",
           "KF378694" :  "MRE from Gigaspora margarita",
           "KP763409" :  "MRE from Gigaspora margarita",
           "KF378704":  "MRE from Gigaspora margarita",
           "FJ984709": "MRE from Funneliformis mosseae",
           "FJ984707": "MRE from Funneliformis mosseae",
           "FJ984704" : "MRE from Funneliformis mosseae",
           "FJ984735" : "MRE from Scuttelospora gilmorei",
         " FJ984646": "MRE from Funneliformis caledonius",
            "FJ984653":"MRE from Funneliformis caledonius",
           "FJ984698":"MRE from Funneliformis caledonius",
           "FJ984654":"MRE from Funneliformis caledonius",
           "FJ984652":"MRE from Funneliformis caledonius",
           "FJ984657": "MRE from Funneliformis caledonius",
           "KU992724": "MRE from Rhizophagus clarus",
           "KU992754":"MRE from Rhizophagus clarus",
           "KU992822":"MRE from Rhizophagus clarus",
           "FJ984724": "MRE from Glomus versiforme",
           "FJ984725": "MRE from Glomus versiforme",
            "KP763367": "MRE from Dentiscutata",
"KP763394": "MRE from Cetraspora pellucida",  
"KM065895": "MRE from Racocetra verrucosa",
 "FJ984643": "MRE from Ambispora fennica",
 "FJ984635": "MRE from Ambispora appendicula",
 "FJ984679": "MRE from Geosiphon pyriformis",
           "KM593997":"MRE from Jimgerdemannia lactiflua",
           "KM594012":"MRE from Jimgerdemannia lactiflua",
           "KM594000":"MRE from Jimgerdemannia lactiflua",
           "KM594005":"MRE from Jimgerdemannia lactiflua",
           "KM594004":"MRE from Jimgerdemannia lactiflua",
         "KM594001": "MRE from Jimgerdemannia flammicorona",
           "KM594003":"MRE from Jimgerdemannia flammicorona",
"KM594011":"MRE from Endogone incrassata",
           "KM594010":"MRE from Endogone incrassata",
         "KM594002":"MRE from Endogone sp.",
         "MG052969": "MRE from Mortierellomycotina sp.",
           "MG052968":  "MRE from Mortierellomycotina sp.",
 "MG052970": "MRE from Mortierellomycotina rostafinskii",
  "MG052966": "MRE from Mortierellomycotina selenospora",    
"MG052967":"MRE from Lobosporangium transversale" ,
       "MG052972":"MRE from Mortierella  gamsii",     
    "MG052973": "MRE from Mortierella  capitata" ,
 "MG052974": "MRE from Mortierella  elongata",
           "MG052971": "MRE from Mortierella  elongata",
"MG052976": "MRE from Mortierella  minutissima",
"MG052975": "MRE from Mortierella  humilis", 
 "NR_044811": "Mycoplasma synoviae",   
 "KC441811": "Bacillus antracis", 
"NR_074620": "Mycoplasma mobile"} 











          

In [38]:
!ls

NebraskMREaln_for_tree.fasta	      ssr_aln.ssu-align.sh
NebraskMREaln_for_tree_renamed.fasta  ssu_aln
ssr_aln				      ssu_aln.ssu-align.sh


In [39]:
output = open("NebraskMREaln_for_tree_renamed.fasta", "w")

for record in SeqIO.parse(open('NebraskMREaln_for_tree.fasta', "rU"), "fasta"):
    acc = record.id.split(".")[0]
    if acc in NewName:
        record.id = NewName[acc] + " " + acc 
        output.write(">"+record.id+"\n"+str(record.seq)+"\n")
    else:
        output.write(">"+record.id+"\n"+str(record.seq)+"\n")
   # print(acc)

In [35]:
!cat NebraskMREaln_for_tree.fasta

>MG052969.1 Uncultured Mollicutes bacterium clone Msp-GBAus27b 16S ribosomal RNA gene, partial sequence
GACGTAGGTGGCGAACGTTATCCGGAATTATTGGGCGTAAAGGGTGCGTAGATGGCTTGG
TAAGTTGGCGGTGGCTCAACCCCGGCAATACTACTAAGCTCGAGCACAGGAGAGGTTAAT
GGAACTCCATGTGGAGCGGTAAAATGCACAGATATATGGAAGAACACCGGCGAAGGCAAT
TAACTATTCTGACTGACATTGAGGCACGAAAGCGTGGGGAGCAAAACGG
>MG052970.1 Uncultured Mollicutes bacterium clone Mros-NRRL A-17819 16S ribosomal RNA gene, partial sequence
GACGTAGGTGGCGAACGTTATCCGGAATTATTGGGCGTAAAGGGTGCGTAGATGGTTTAG
TAAGTTGGCGGTGGCTCAACCCCGACAATACTACTAAACTCGAGTACAGGAGAGGTTAAT
GGAACTCCATGTGGAGCGGTAAAATGCACAGATATATGGAAGAACACCGGCGAAGGCAAT
TAACTATTCTGACTGACATTGAGGCACGAAAGCGTGGGGAGCAAAACGG
>MG052968.1 Uncultured Mollicutes bacterium clone Msp-AD185 16S ribosomal RNA gene, partial sequence
GACGTAGGTGGCGAACGTTATCCGGAATTATTGGGCGTAAAGGGTGCGTAGATGGCTTGG
TAAGTTGGCGGTGGCTCAACCCCGGCAATACTACTAAGCTCGAGTACAGGAGAGGTTAAT
GGAACTCCATGTGGAGCGGTAAAATGCATAGATATATGGAAGAACACCGGCGAAGGCAAT
TAACTATTCTGACTGACATTGAGGCACGAAAGCGTGG

In [40]:
!cat NebraskMREaln_for_tree_renamed.fasta

>MRE from Mortierellomycotina sp. MG052969
GACGTAGGTGGCGAACGTTATCCGGAATTATTGGGCGTAAAGGGTGCGTAGATGGCTTGGTAAGTTGGCGGTGGCTCAACCCCGGCAATACTACTAAGCTCGAGCACAGGAGAGGTTAATGGAACTCCATGTGGAGCGGTAAAATGCACAGATATATGGAAGAACACCGGCGAAGGCAATTAACTATTCTGACTGACATTGAGGCACGAAAGCGTGGGGAGCAAAACGG
>MRE from Mortierellomycotina rostafinskii MG052970
GACGTAGGTGGCGAACGTTATCCGGAATTATTGGGCGTAAAGGGTGCGTAGATGGTTTAGTAAGTTGGCGGTGGCTCAACCCCGACAATACTACTAAACTCGAGTACAGGAGAGGTTAATGGAACTCCATGTGGAGCGGTAAAATGCACAGATATATGGAAGAACACCGGCGAAGGCAATTAACTATTCTGACTGACATTGAGGCACGAAAGCGTGGGGAGCAAAACGG
>MRE from Mortierellomycotina sp. MG052968
GACGTAGGTGGCGAACGTTATCCGGAATTATTGGGCGTAAAGGGTGCGTAGATGGCTTGGTAAGTTGGCGGTGGCTCAACCCCGGCAATACTACTAAGCTCGAGTACAGGAGAGGTTAATGGAACTCCATGTGGAGCGGTAAAATGCATAGATATATGGAAGAACACCGGCGAAGGCAATTAACTATTCTGACTGACATTGAGGCACGAAAGCGTGGGGAGCAAAACGG
>MRE from Mortierellomycotina selenospora MG052966
GACATAGGTGGCGAACGTTATCCGGAATTATTGGGCGTAAAGGGTGCGTAGATCGTGCAATAAGTTATCGGTGGCCCAACCTCGGTAATACTGTTGCACTAGAGTTCAGGGGAGGTTGATG

# Inferring and rooting the tree

In [41]:
aln = LoadSeqs('NebraskMREaln_for_tree_renamed.fasta', moltype=DNA)
t_unroot = build_tree_from_alignment(aln, moltype=DNA)

In [42]:
!grep "KC441811" NebraskMREaln_for_tree_renamed.fasta

>Bacillus antracis KC441811


In [43]:
t_rooted = t_unroot.rootedWithTip("Bacillus antracis KC441811")

In [44]:
t_rooted.writeToFile('NebraskaMRE.tree')